In [0]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from google.colab import files


def get_search_results_df(keyword):
    columns = ['rank','title','url', 'affiliate_url']
    df = pd.DataFrame(columns=columns)
    html_doc = requests.get('https://www.google.com/search?q=' +keyword).text
    soup = BeautifulSoup(html_doc, 'html.parser') # BeautifulSoupの初期化
    tags = soup.find_all('div',{'class': 'ZINbbc xpd O9g5cc uUPGi'})
    rank = 1
    for tag in tags:
        # タイトルとサブテキストがBNeaweクラスにあるので
        # find_allでリスト化→[0]でタイトルテキストを持ってくる
        title = tag.find_all('div', {'class': 'BNeawe'})[0].string 
        url = query_string_remove(tag.a.get('href').replace('/url?q=', ''))
        affiliate_url = get_a8_links(url)
        se = pd.Series([rank, title, url, affiliate_url], columns)
        df = df.append(se, ignore_index=True)
        rank += 1
        if rank == 10 + 1:
            break
    return df
 

def query_string_remove(url):
    return url[:url.find('&')]

def get_a8_links(url):
    html_doc = requests.get(url).text
    soup = BeautifulSoup(html_doc, 'html.parser')
    tags = soup.select('a')
    urls = ''
    for tag in tags:
        try:
            url = tag.get('href')
            if url.find('https://px.a8.net') > -1:
                urls += url + '\n'
        except Exception as e:
            continue
    return urls

keyword = "非リア　モテない" # ここに検索するキーワードを入力
search_results_df = get_search_results_df(keyword)
search_results_df.head(15)

filename = "affi_link.csv"
search_results_df.to_csv(filename, encoding = 'utf-8-sig')
files.download(filename)
